In [17]:
from math import log
import operator

In [18]:
def createDataSet():
    dataSet = [[1,1,'maybe'],
               [1,1,'yes'],
               [1,0,'no'],
               [0,1,'no'],
               [0,1,'no']]
    labels = ['no surfacing','flippers']
    return dataSet,labels

计算熵,dataSet为list，里面的每条数据也都是list

In [19]:
def calcShannonEnt(dataSet): 
    numEntries = len(dataSet) #dataSet的样本量
    labelCounts = {}   #dict
    for featVec in dataSet:   #featVec也是list
        currentLabel = featVec[-1]  #每条数据最后一列为label  eg:[1,0,'yes']
        if currentLabel not in labelCounts.keys():   
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1    #分别计算所有类别的数据个数
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key] / numEntries)
        shannonEnt -= prob * log(prob,2)  #熵
    return shannonEnt  

按照给定特征划分数据集，dataSet为待划分数据集，axis为划分数据集的特征，value为需要返回的特征的值

In [20]:
def splitDataSet (dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet :
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]  #从头开始取到第axis-1个元素
            reducedFeatVec.extend(featVec[axis+1:])  #把从第axis+1个到最后一个元素插入reducedFeatVec中
            retDataSet.append(reducedFeatVec) #将reducedFeatVec作为一个list插入retDataSet中
    return retDataSet

选择最好的数据集划分方式

In [21]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 #特征的个数
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]  #所有数据第i个特征的取值
        uniqueVals = set(featList)  #第i的特征的可能取值 set中元素不可重复
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)  #按照第i个特征的某个取值value划分数据集
            prob = len(subDataSet)/float(len(dataSet))  #特征i的取值等于value的数据所占概率
            newEntropy += prob * calcShannonEnt(subDataSet) #按照特征i划分之后的熵
        infoGain = baseEntropy - newEntropy #信息增益
        if (infoGain > bestInfoGain):  #计算最大的信息增益
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

采用多数表决的方法决定叶子结点中数据的分类，classList为同一叶子结点中的数据的类别

In [22]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse = True) 
    #将classCount分解为元祖，按照第2个元素即classCount进行排序，逆序：从大到小
    return sortedClassCount[0][0]

创建树，dataSet，labels为数据集中所有特征的标签

In [23]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet] #数据集的所有类标签
    if classList.count(classList[0]) == len(classList):  #count(classList[0])为统计classList[0]出现的次数
        return classList[0]  #如果dataSet中所有的类标签完全相同则直接返回该类标签
    if len(dataSet[0]) == 1:  #dataSet第一条数据中只剩标签，即已经使用完了所有特征
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet) #bestFeat返回最佳分类特征的索引
    bestFeatLabel = labels[bestFeat]  #bestFeat的标签
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet] #数据中该特征的所有取值
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]  #labels[bestfeat]已经被删除
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels) #递归创建子树
    return myTree 

In [24]:
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)  #查找当前列表中第一个匹配 firstStr的元素
    for key in secondDict.keys():
        if testVec[featLabelsIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel =classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel

In [32]:
def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)

In [9]:
try:    
    !jupyter nbconvert --to python trees.ipynb
    # python即转化为.py，script即转化为.html
    # file_name.ipynb即当前module的文件名
except:
    pass

[NbConvertApp] Converting notebook trees.ipynb to python
[NbConvertApp] Writing 4283 bytes to trees.py


In [26]:
#dataSet,labels = createDataSet()
#chooseBestFeatureToSplit(dataSet)
#myTree = createTree(dataSet,labels)

In [34]:
#storeTree(myTree,'classifierStorage.txt')
#grabTree('classifierStorage.txt')